In [158]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine

In [2]:
today = datetime.today()
yyyy = today.year
mm = today.month
print(yyyy, mm)

2023 9


In [3]:
year = list(range(2023, 2022, -1))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)


[2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [132]:
# book_list에서 책 1권의 정보를 추출하는 함수
def extract_bookinfo(yy, mon, book_list):
    for book in book_list:
        title_h = book.select_one(".gd_res").get_text()
        title_f = book.select_one(".gd_nameF").get_text() if book.select_one(".gd_nameF") != None else ""
        title_m = book.select_one(".gd_name").get_text() if book.select_one(".gd_name") != None else "제목없음"
        title_e = book.select_one(".gd_nameE").get_text() if book.select_one(".gd_nameE") != None else ""
        detail_link = "https://www.yes24.com" + book.select_one(".gd_name")["href"] if book.select_one(".gd_name") != None else "제목없음"
        author = [i.get_text() for i in book.select(".authPub.info_auth > a") if book.select(".authPub.info_auth > a") != None]
        publisher = book.select_one(".authPub.info_pub > a").get_text() if book.select_one(".authPub.info_pub > a") != None else ""
        pub_date = book.select_one(".authPub.info_date").get_text() if book.select_one(".authPub.info_date") != None else ""
        price = book.select_one(".yes_b").get_text() if book.select_one(".yes_b") != None else 0
        n_reviews = book.select_one(".txC_blue").get_text() if book.select_one(".txC_blue") != None else 0
        review_link = book.select_one(".rating_rvCount > a")['href'] if book.select_one(".rating_rvCount > a") != None else ""
        rating = book.select_one("span.rating_grade > .yes_b").get_text() if book.select_one("span.rating_grade > .yes_b") != None else 0
        tags = [i.get_text() for i in book.select(".info_row.info_tag > .tag > a") if book.select(".info_row.info_tag > .tag > a") != None]
        return [yy, mon, title_h, title_f, title_m, title_e, detail_link, author, publisher, pub_date, price, n_reviews,
             review_link, rating, tags]
     

In [161]:
def detail_page_info(urls):
    detail_result = pd.DataFrame()
    for index, url2 in enumerate(urls[:10]):
        print(f"{index}/{len(urls)} 데이터 추출 중")
        r2 = requests.get(url2)
        soup2 = bs(r2.text, 'lxml')
        book_id = url2.split("/")[-1]
        if soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td") != None:
            if "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")) == 2 and \
                       soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")[1][-2:] == "mm":
                    page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(2) > td").text.split("|")
                    weight = 0
            elif "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")) == 2 and \
                   soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")[1][-2:] == "mm":
                    page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(3) > td").text.split("|")
                    weight = 0
            elif "쪽수" in soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > th").text:
                if len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")) == 3:
                    page, weight, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")
                elif len(soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")) == 2 and \
                       soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")[1][-2:] == "mm":
                        page, size = soup2.select_one("div.infoSetCont_wrap tr:nth-child(4) > td").text.split("|")
                        weight = 0
        else:
            page = 0
            weight = 0
            size = ""
        category = list({i. text for i in soup2.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") if soup2.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") != None})
        book_intro = soup2.select_one(".infoWrap_txtInner").get_text() if soup2.select_one(".infoWrap_txtInner") != None else ""
        pub_book_intro = soup2.select_one(".infoWrap_txt").get_text() if soup2.select_one(".infoWrap_txt") != None else ""

        result_detail = [book_id, page, weight, size, category, book_intro, pub_book_intro]
        colname = ['book_id', 'page', 'weight', 'size', 'category', 'book_intro', 'pub_book_intro']
        temp = pd.DataFrame([result_detail], columns = colname)
        detail_result = pd.concat([detail_result, temp])
        detail_result.to_csv("./data/yes24_2023_01best_detail.csv", index=False)
    return detail_result

In [163]:
def to_db(df):
    for col in df.columns:
        df[col] = df[col].apply(str)    
    engine = create_engine('sqlite:///yes24_final.db', echo= False)
    conn = engine.raw_connection()
    df.to_sql('yes24_final_fn', con=conn, if_exists='append')
    print("데이터베이스 저장 완료")

In [164]:
result = []
for yy in year:
    for mon in month if yy != yyyy else range(1, mm)[:1]:
        for p in page[:2]:
            print(f"{yy}년{mon}월{p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber='001', pageNumber=p, pageSize=120, type='month', saleYear=yy, saleMonth=mon)
            
            try:
                r = requests.get(url, params=payload) # yes24 서버에 url로 자료 요청
            except Exception as e:
                print(e)
            
            soup = bs(r.text, 'lxml')   # yes24 서버에서 받은 응답 값 텍스트를 beautifulsoup을 통해 html로 파싱
            book_list = soup.select("#yesBestList > li")
            result.extend(extract_bookinfo(yy, mon, book_list))  # extract_bookinfo 함수로 추출한 데이터를 result에 추가
            time.sleep(5)
df2 = pd.DataFrame(result, columns=['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', \
                                    'author', 'publisher', 'pub_date', 'price', 'n_reviews', \
                                     'review_link', 'rating', 'tags'])
df2.to_csv("./data/new_yes24bestseller2023_01_1000.csv", index=False)
data2 = pd.read_csv("./data/new_yes24bestseller2023_01_1000.csv")
urls = data2['detail_link']
detail_page_data = detail_page_info(urls)  # detail_page_info 함수로 상세페이지 데이터 추출
df2['book_id'] = df2['detail_link'].apply(lambda x: x.split("/")[-1]) 
final_result_df = pd.merge(df2, detail_page_data, how='inner', on="book_id") # 베스트 셀러 목록과 detail info 합침
final_result_df.head(2)
to_db(final_result_df) # 최종 데이터를 sqlite3 데이터 베이스에 저장

0/240 데이터 추출 중중
1/240 데이터 추출 중
2/240 데이터 추출 중
3/240 데이터 추출 중
4/240 데이터 추출 중
5/240 데이터 추출 중
6/240 데이터 추출 중
7/240 데이터 추출 중
8/240 데이터 추출 중
9/240 데이터 추출 중
데이터베이스 저장 완료


C:\Users\hi\AppData\Local\Temp\ipykernel_13356\1682931616.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('yes24_final_fn', con=conn, if_exists='append')


,yy,mon,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,...,review_link,rating,tags,book_id,page,weight,size,category,book_intro,pub_book_intro
0,2023,1,[도서],,긴긴밤,,https://www.yes24.com/Product/Goods/97255028,[루리],문학동네,2021년 02월,...,https://www.yes24.com/Product/Goods/97255028?R...,9.8,"[#대장금_출판사대표추천도서, #독서지도안, #북클러버의선택, #2022아침독서추천...",97255028,144쪽,340g,153*220*11mm,"[5-6학년 창작동화, 어린이 문학, 창작동화, 5-6학년 그림/동화책, 5-6학년...",\n\r\n제21회 문학동네어린이문학상 대상 수상작 『긴긴밤』\r\n세상에 마지막 ...,\n\n\r\n제21회 문학동네어린이문학상 대상 수상작 『긴긴밤』\r\n세상에 마지...
1,2023,1,[도서],,하루 한 장 고전 수업,365일 인생의 내공을 기르는,https://www.yes24.com/Product/Goods/115382717,[조윤제],비즈니스북스,2022년 11월,...,https://www.yes24.com/Product/Goods/115382717?...,9.6,[],115382717,408쪽,708g,152*225*24mm,"[처세술/삶의 자세, 교양으로 읽는 인문, 인문/교양, 자기계발, 국내도서, 인문]",\n\r\n365가지 고전의 내공을 내 것으로 만들어라!\r\n대한민국이 사랑하는 ...,\n\n\r\n365가지 고전의 내공을 내 것으로 만들어라!\r\n대한민국이 사랑하...


In [134]:
len(result)

2

In [13]:
colnames = ['yy', 'mon', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', 'author', 'publisher',
            'pub_date', 'price', 'n_reviews', 'review_link', 'rating', 'tags']

In [14]:
df = pd.DataFrame(result, columns= colnames)

In [15]:
df.to_csv("./data/new_yes24bestseller2023_01_1000.csv", index=False)

In [16]:
data = pd.read_csv("./data/new_yes24bestseller2023_01_1000.csv")

In [20]:
urls = data['detail_link']

In [ ]:
# 데이터 베이스에 저장하기
# 파일로 바로 저장하기

상세 페이지 수집

In [107]:
detail_result.columns

Index(['book_id', 'page', 'weight', 'size', 'category', 'book_intro',
       'pub_book_intro'],
      dtype='object')

In [121]:
detail_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 0
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   book_id         1000 non-null   object
 1   page            1000 non-null   object
 2   weight          1000 non-null   object
 3   size            1000 non-null   object
 4   category        1000 non-null   object
 5   book_intro      1000 non-null   object
 6   pub_book_intro  1000 non-null   object
dtypes: object(7)
memory usage: 62.5+ KB


# 데이터 베이스에 자료 저장하기
* pandas 데이터 프레임을 DB에 바로 저장하기

In [105]:
import sqlite3

In [106]:
con = sqlite3.connect("sqlite3_yes24_best2023_01.db")
cursor = con.cursor()

In [112]:
cursor.execute(f'CREATE TABLE IF NOT EXISTS yes24_best_202301({list(detail_result.columns)})')
con.commit()

In [116]:
detail_result['page'] = detail_result['page'].apply(str)

In [117]:
detail_result['weight'] = detail_result['weight'].apply(str)

In [122]:
detail_result['category'] = detail_result['category'].apply(str)

In [123]:
detail_result.to_sql('yes24_best_202301', con, if_exists='append', index=False)

1000

In [126]:
cursor.execute(
    '''
    SELECT * FROM yes24_best_202301 limit 10;
    ''')
for row in cursor.fetchall():
    print(row)

('115142458', '280쪽 ', ' 408g ', ' 145*210*18mm', "['노년 / 죽음', '국내도서', '주제로 읽는 인문학 ', '인문']", '\n\r\n“하나의 문이 닫히면 또 하나의 문이 열린다. \r\n그러니 더 이상 고민하지 말고 그냥 재미있게 살아라!”\r\n\r\n30년 동안 정신분석 전문의로 일해 온 김혜남이 벌써 마흔이 된 당신에게 해 주고 싶은 말들을 담은 책이다. 그녀는 지금껏 살면서 한 가지 후회하는 게 있다면 스스로를 너무 닦달하며 인생을 숙제처럼 산 것이라고 말한다. 의사로, 엄마로, 아내로, 며느리로, 딸로 살면서 늘 의무와 책임감에 치여 어떻게든 그 모든 역할을 잘해 내려 애썼고 그러다 보니 정작 누려야 할 삶의 즐거움들을 놓쳐 버렸다는 것이다. 그러다 22년 전 마흔세 살에 파킨슨병 진단을 받으며 비로소 알게 되었다. 자신이 없으면 집안도 병원도 제대로 안 굴러갈 것 같았는데 세상은 너무나 멀쩡히 잘 굴러갔다. 그제야 그녀는 자신의 곁을 지켜 주는 사람들을 다시 보게 되었고, 놓쳐서는 안 될 인생의 소중한 것들이 무엇인지를 돌아보게 되었다. 그래서 벌써 마흔이 되어 버린, 후회 없는 인생을 살고 싶은 사람들에게 말해 주고 싶은 것도 딱 한가지다. “하나의 문이 닫히면 또 하나의 문이 열린다. 그러니 더 이상 고민하지 말고 그냥 재미있게 살아라!” \r\n\r\n이 책은 2015년 ‘오늘 내가 사는 게 재미있는 이유’라는 제목으로 출간, 독자들의 사랑을 받으며 10만 부 돌파 베스트셀러가 된 것을 기념해 펴낸 스페셜 에디션으로 저자가 30년간 정신분석 전문의로 일하며 깨달은 인생의 비밀과 22년간 파킨슨병을 앓으면서도 유쾌하게 살 수 있는 이유를 전한다. 또한 ‘완벽한 때는 결코 오지 않는 법이다’, ‘때론 버티는 것이 답이다’, ‘제발 모든 것을 상처라고 말하지 말 것’, ‘가까운 사람일수록 해서는 안 될 것들이 있다’ 등 환자들에게 미처 하지 못했던 그러나 꼭 해 주고 싶은 이야기들을 담고 있다. \r\

# sqlalchemy를 이용하는 방법

C:\Users\hi\AppData\Local\Temp\ipykernel_13356\3373494623.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  detail_result.to_sql('yes24_best_202301_sqlalchemy', con=conn, if_exists='append')


1000

In [130]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite3_yes24_best2023_01_sqlalchemy.db', echo= False)
conn = engine.raw_connection()
test = pd.read_sql('select * from yes24_best_202301_sqlalchemy limit 3', conn)
test
conn.close()

C:\Users\hi\AppData\Local\Temp\ipykernel_13356\3687058998.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql('select * from yes24_best_202301_sqlalchemy limit 3', conn)


In [131]:
test

,index,book_id,page,weight,size,category,book_intro,pub_book_intro
0,0,115142458,280쪽,408g,145*210*18mm,"['노년 / 죽음', '국내도서', '주제로 읽는 인문학 ', '인문']",\n\r\n“하나의 문이 닫히면 또 하나의 문이 열린다. \r\n그러니 더 이상 고...,\n\n\r\n“하나의 문이 닫히면 또 하나의 문이 열린다. \r\n그러니 더 이상...
1,0,116579452,280쪽,470g,145*210*17mm,"['자녀교육일반', '가정 살림', '자녀교육', '국내도서', '학습법일반']",\n\r\n17만 유튜브 [분당강쌤]이 공개한 대입까지 직행하는 공부의 기본기\r\...,\n\n\r\n17만 유튜브 [분당강쌤]이 공개한 대입까지 직행하는 공부의 기본기\...
2,0,116046489,376쪽,558g,136*200*30mm,"['에세이', '명사/연예인 에세이 ', '국내도서', '한국 에세이']","\n\r\n우리들의 배우 김혜자의 연기, 인생“그럼에도 불구하고, 생에 감사해”한국...","\n\n\r\n우리들의 배우 김혜자의 연기, 인생“그럼에도 불구하고, 생에 감사해”..."


In [ ]:
def ramentop20todb(df, today):
    engine = create_engine("mysql+mysqldb://<id>:"+"<password>"+"@59.7.246.88:3306/ramen", encoding='utf-8')
    conn = engine.connect()
    try:
        df.to_sql(name='Top20s{}'.format(today), con=engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    conn.close()
    return print("DB saving has been completed")

In [ ]:
detail_info = pd.DataFrame()
for url2 in tqdm(urls):
    r = requests.get(url2, stream=True)
    soup = bs(r.text, 'lxml')
    book_id_num = url2.split("/")[-1]
    book_pws = soup.select_one("tbody > tr:nth-child(2) > td.txt.lastCol").get_text().split("|") if soup.select_one("tbody > tr:nth-child(2) > td.txt.lastCol") != None  else ""
    if len(book_pws) < 3:
        page = 0
        weight = 0
        size = 0
    else:
        page, weight, size = book_pws
    category = list({i.get_text() for i in soup.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") if soup.select("div.infoSetCont_wrap > dl:nth-child(1) > dd > ul a") != None})
    abstract = soup.select_one(".infoWrap_txtInner").get_text() if soup.select_one(".infoWrap_txtInner") != None else ""
    pub_review = soup.select_one("#infoset_pubReivew div.infoWrap_txt").get_text() if soup.select_one("#infoset_pubReivew div.infoWrap_txt") != None else ""
    result = pd.DataFrame([[book_id_num, page, weight, size, category, abstract, pub_review]], columns=['book_id_num','page', 'weight', 'size', 'category', 'abstract', 'pub_review'])
    detail_info = pd.concat([detail_info, result])
    time.sleep(5)

In [ ]:
detail_info

In [ ]:
pd.concat()